# Feature Engineering

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
all_games = pd.read_csv("/Users/tativalentine/Documents/GitHub/games_clean.csv", index_col="id")

In [4]:
all_games.shape

(97291, 17)

In [5]:
all_games.head()

,date,home_team_score,period,postseason,season,status,visitor_team_score,home_team.id,home_team.abbreviation,home_team.conference,home_team.division,home_team.full_name,visitor_team.id,visitor_team.abbreviation,visitor_team.conference,visitor_team.division,visitor_team.full_name
id,,,,,,,,,,,,,,,,,
47179,2019-01-30T00:00:00.000Z,126,4,False,2018,Final,94,2,BOS,East,Atlantic,Boston Celtics,4,CHA,East,Southeast,Charlotte Hornets
48751,2019-02-09T00:00:00.000Z,112,4,False,2018,Final,123,2,BOS,East,Atlantic,Boston Celtics,13,LAC,West,Pacific,LA Clippers
48739,2019-02-08T00:00:00.000Z,117,4,False,2018,Final,110,23,PHI,East,Atlantic,Philadelphia 76ers,8,DEN,West,Northwest,Denver Nuggets
48740,2019-02-08T00:00:00.000Z,119,4,False,2018,Final,106,30,WAS,East,Southeast,Washington Wizards,6,CLE,East,Central,Cleveland Cavaliers
48746,2019-02-08T00:00:00.000Z,102,4,False,2018,Final,96,26,SAC,West,Pacific,Sacramento Kings,16,MIA,East,Southeast,Miami Heat


In [6]:
# Feature engineering

g = all_games.copy()

g["date"] = pd.to_datetime(all_games["date"]).dt.tz_localize(None)

# creating target variable
g["winner"] = np.where(g["home_team_score"] > g["visitor_team_score"], 1, 0)

# creating avg points by team across all years (not sure how useful this will be)
home_avg_pts_map = g[["home_team.id", "home_team_score"]].groupby("home_team.id").mean().squeeze()
g["home_team_avg_score_historical"] = g["home_team.id"].map(home_avg_pts_map).round(1)

visitor_avg_pts_map = g[["visitor_team.id", "visitor_team_score"]].groupby("visitor_team.id").mean().squeeze()
g["visitor_team_avg_score_historical"] = g["visitor_team.id"].map(visitor_avg_pts_map).round(1)

In [7]:
# creating an identifier for the team in a specific season
g["home_team_id_year"] = (g["home_team.id"].astype(str) + " " + g["season"].astype(str)).values
g["visitor_team_id_year"] = (g["visitor_team.id"].astype(str) + " " + g["season"].astype(str)).values

In [8]:
# creating average pts feature
# average pts scored per game in that season
# based on if they are home or away because there is a statistically significant difference
home_avg_score_map = g[["home_team_id_year", "home_team_score"]].groupby(["home_team_id_year"]).mean().squeeze()
visitor_avg_score_map = g[["visitor_team_id_year", "visitor_team_score"]].groupby(["visitor_team_id_year"]).mean().squeeze()

g["home_team_avg_score"] = g["home_team_id_year"].map(home_avg_score_map).round(1)
g["visitor_team_avg_score"] = g["visitor_team_id_year"].map(visitor_avg_score_map).round(1) 

In [9]:
# create an average difference in pts feature
# will calculate on average how much a team wins or loses by
# this will hopefully benefit lower scoring teams that also play defensively
avg_score_diff = g[["home_team_id_year", "home_team_avg_score", "visitor_team_avg_score"]].groupby("home_team_id_year").mean()
avg_score_diff = avg_score_diff["home_team_avg_score"] - avg_score_diff["visitor_team_avg_score"]
g["home_avg_score_diff"] = g["home_team_id_year"].map(avg_score_diff)

avg_score_diff = g[["visitor_team_id_year", "visitor_team_avg_score", "home_team_avg_score"]].groupby("visitor_team_id_year").mean()
avg_score_diff = avg_score_diff["visitor_team_avg_score"] - avg_score_diff["home_team_avg_score"]
g["visitor_avg_score_diff"] = g["visitor_team_id_year"].map(avg_score_diff)

In [10]:
g

,date,home_team_score,period,postseason,season,status,visitor_team_score,home_team.id,home_team.abbreviation,home_team.conference,home_team.division,home_team.full_name,visitor_team.id,visitor_team.abbreviation,visitor_team.conference,visitor_team.division,visitor_team.full_name,winner,home_team_avg_score_historical,visitor_team_avg_score_historical,home_team_id_year,visitor_team_id_year,home_team_avg_score,visitor_team_avg_score,home_avg_score_diff,visitor_avg_score_diff
id,,,,,,,,,,,,,,,,,,,,,,,,,,
47179,2019-01-30,126,4,False,2018,Final,94,2,BOS,East,Atlantic,Boston Celtics,4,CHA,East,Southeast,Charlotte Hornets,1,105.5,98.0,2 2018,4 2018,113.3,108.4,4.026829,-3.791892
48751,2019-02-09,112,4,False,2018,Final,123,2,BOS,East,Atlantic,Boston Celtics,13,LAC,West,Pacific,LA Clippers,0,105.5,100.3,2 2018,13 2018,113.3,112.7,4.026829,0.223457
48739,2019-02-08,117,4,False,2018,Final,110,23,PHI,East,Atlantic,Philadelphia 76ers,8,DEN,West,Northwest,Denver Nuggets,1,102.9,104.0,23 2018,8 2018,117.9,107.7,8.839759,-4.969412
48740,2019-02-08,119,4,False,2018,Final,106,30,WAS,East,Southeast,Washington Wizards,6,CLE,East,Central,Cleveland Cavaliers,1,103.1,98.2,30 2018,6 2018,116.4,103.6,7.187013,-8.797368
48746,2019-02-08,102,4,False,2018,Final,96,26,SAC,West,Pacific,Sacramento Kings,16,MIA,East,Southeast,Miami Heat,1,105.3,95.6,26 2018,16 2018,114.8,105.3,4.853247,-6.616216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473801,2021-12-11,112,4,False,2021,Final,127,27,SAS,West,Southwest,San Antonio Spurs,8,DEN,West,Northwest,Denver Nuggets,0,105.6,104.0,27 2021,8 2021,113.0,108.1,5.469231,-0.170370
473863,2021-12-19,137,4,False,2021,Final,106,24,PHX,West,Pacific,Phoenix Suns,4,CHA,East,Southeast,Charlotte Hornets,1,108.2,98.0,24 2021,4 2021,111.5,113.6,4.622727,4.932258
473890,2021-12-23,107,4,False,2021,Final,115,8,DEN,West,Northwest,Denver Nuggets,4,CHA,East,Southeast,Charlotte Hornets,0,109.4,98.0,8 2021,4 2021,111.5,113.6,3.431034,4.932258


In [11]:
g.to_csv("/Users/tativalentine/Documents/GitHub/games_with_features.csv")

# Stats data
#### This data is used to build the model

In [12]:
stats = pd.read_csv("/Users/tativalentine/Documents/GitHub/stats_clean.csv", index_col="id")

In [15]:
stats

,ast,blk,dreb,fg3_pct,fg3a,fg3m,fg_pct,fga,fgm,ft_pct,fta,ftm,min,oreb,pf,pts,reb,stl,turnover,game.id,game.date,game.home_team_id,game.home_team_score,game.season,game.visitor_team_id,game.visitor_team_score,team.id,winner
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1069008,0.0,1.0,2.0,0.200,5.0,1.0,0.333,9.0,3.0,0.000,0.0,0.0,0 days 00:20:08,0.0,0.0,7.0,2.0,0.0,2.0,45237,2019-01-17,4,114,2018,26,95,26,1
1069009,4.0,0.0,5.0,0.000,2.0,0.0,0.200,5.0,1.0,0.000,0.0,0.0,0 days 00:19:22,1.0,1.0,2.0,6.0,0.0,0.0,45237,2019-01-17,4,114,2018,26,95,26,1
1069010,4.0,1.0,5.0,0.000,0.0,0.0,0.667,6.0,4.0,0.000,0.0,0.0,0 days 00:27:24,6.0,2.0,8.0,11.0,3.0,2.0,45237,2019-01-17,4,114,2018,26,95,26,1
1069011,1.0,0.0,1.0,0.545,11.0,6.0,0.500,18.0,9.0,0.000,0.0,0.0,0 days 00:32:06,0.0,2.0,24.0,1.0,2.0,0.0,45237,2019-01-17,4,114,2018,26,95,26,1
1069012,8.0,1.0,5.0,0.000,2.0,0.0,0.400,10.0,4.0,0.667,3.0,2.0,0 days 00:30:30,0.0,4.0,10.0,5.0,1.0,4.0,45237,2019-01-17,4,114,2018,26,95,26,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293372,0.0,0.0,1.0,0.000,0.0,0.0,0.286,7.0,2.0,0.667,3.0,2.0,0 days 00:21:10,0.0,3.0,6.0,1.0,0.0,0.0,13561,2001-03-20,6,75,2000,20,110,6,0
293373,5.0,0.0,0.0,0.667,3.0,2.0,0.500,8.0,4.0,1.000,2.0,2.0,0 days 00:14:54,0.0,2.0,12.0,0.0,0.0,1.0,13561,2001-03-20,6,75,2000,20,110,6,0
293374,0.0,0.0,0.0,0.000,0.0,0.0,0.000,1.0,0.0,0.000,0.0,0.0,0 days 00:06:29,0.0,0.0,0.0,0.0,0.0,1.0,13561,2001-03-20,6,75,2000,20,110,6,0


#### Convert the date column to datetime format so the data can be sorted by date.

#### Convert the minutes column to timedelta so that they can be added together. (The rows of data are stats put up by a specific player in a specific game. These stats from these rows will be aggregated... grouping by team and game. This will result in the stats that the TEAM put up for that game, which will be useful because we are predicting which TEAM will win the game.)

In [13]:
# Convert game date to datetime
stats["game.date"] = pd.to_datetime(stats["game.date"]).dt.tz_localize(None)

# Convert string to timedelta
stats["min"] = [pd.Timedelta(minutes=int(time[0]), seconds=int(time[1])) for time in stats["min"].str.split(":").values]

# Create target variable & a "labels" dataframe
stats["winner"] = np.where(stats["game.home_team_score"].values > stats["game.visitor_team_score"].values, 1,0)
labels = stats[["game.id", "game.date", "game.season", "winner"]].groupby("game.id").first()

#### This code block aggregates the individual player statistics into team statistics

In [17]:
# split into 2 dataframes so that grouping data by game id doesn't
# group players of opposing teams
home_stats = stats[stats["team.id"].eq(stats["game.home_team_id"])]
away_stats = stats[stats["team.id"].eq(stats["game.visitor_team_id"])]

# define how to aggregate statistics when grouping player stats
agg_map = {"ast": "sum", 
           "blk": "sum", 
           "dreb": "sum", 
           "fg3_pct": "mean", 
           "fg3a": "sum", 
           "fg3m": "sum", 
           "fg_pct": "mean",
          "fga": "sum",
          "fgm": "sum",
          "ft_pct": "mean",
          "fta": "sum",
          "ftm": "sum",
          "min": "sum",
          "oreb": "sum",
          "pf": "sum",
          "pts": "sum",
          "reb": "sum",
          "stl": "sum",
          "turnover": "sum",
          "game.id": "first",
          "game.date": "first",
          "game.season": "first",
          "game.home_team_id": "first",
          "game.home_team_score": "first",
          "game.visitor_team_id": "first",
          "game.visitor_team_score": "first",
          "team.id": "first",}

# aggregate player stats to team stats
home_games = home_stats.groupby("game.id").agg(agg_map)
away_games = away_stats.groupby("game.id").agg(agg_map)


#### To predict which team will win a game, the model will look at the average stats that a team put up in their 20 most recent games.* (called the rolling average from here on out)

#### Their 20 most recent home games if the game being predicted is a home game Their 20 most recent away games if the game being predicted is an away game

In [18]:
# sort rows by team and date to prepare for the rolling average
home_games = home_games.sort_values(["game.home_team_id", "game.date"])
away_games = away_games.sort_values(["game.visitor_team_id", "game.date"])

In [19]:
# Only use the columns that refer to stats when calculating the rolling average
stats_cols = ["ast","blk","dreb","fg3_pct","fg3a","fg3m","fg_pct","fga","fgm","ft_pct","fta","ftm","oreb",
              "pf","pts","reb","stl","turnover"]

rolling_stats_home = pd.DataFrame()
for team in home_games["game.home_team_id"].unique():                       # for each team
    games = home_games[home_games["game.home_team_id"].eq(team)]            # get only home games for that team
    games_stats = games[stats_cols]                                         # Only use the columns that refer to stats when calculating the rolling average
    rolling_stats_home = rolling_stats_home.append(games_stats.rolling(window=20).mean())  # calculate the rolling average (20 game average)
    rolling_stats_home = rolling_stats_home.shift()                         # make the rolling average not include the current game
rolling_stats_home.dropna(inplace=True)


rolling_stats_away = pd.DataFrame()
for team in away_games["game.visitor_team_id"].unique():
    games = away_games[away_games["game.visitor_team_id"].eq(team)]
    games_stats = games[stats_cols]
    rolling_stats_away = rolling_stats_away.append(games_stats.rolling(window=20).mean())
    rolling_stats_away = rolling_stats_away.shift()  # make the rolling average not include the current game
rolling_stats_away.dropna(inplace=True)

/var/folders/nf/rr1y3sz51_d8gbgc735ltjtc0000gn/T/ipykernel_19696/3018443371.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rolling_stats_home = rolling_stats_home.append(games_stats.rolling(window=20).mean())  # calculate the rolling average (20 game average)
/var/folders/nf/rr1y3sz51_d8gbgc735ltjtc0000gn/T/ipykernel_19696/3018443371.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rolling_stats_home = rolling_stats_home.append(games_stats.rolling(window=20).mean())  # calculate the rolling average (20 game average)
/var/folders/nf/rr1y3sz51_d8gbgc735ltjtc0000gn/T/ipykernel_19696/3018443371.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rolling_stats_home = rolling_stats_home.append(games_stats.rolling(window=2

In [20]:
# Rename columns in the home and away dataframes
rolling_stats_home.columns = ["home_" + col_name for col_name in rolling_stats_home.columns]
rolling_stats_away.columns = ["away_" + col_name for col_name in rolling_stats_away.columns]
# Now that columns are renamed they can be merged into the same dataframe
rolling_stats = pd.merge(rolling_stats_home, rolling_stats_away, on="game.id")

In [21]:
# NOTE: adding columns to the start of the dataframe will mess up this code
# adding columns to the end will not
rolling_stats_diff = pd.DataFrame(index=rolling_stats.index)
home_cols = rolling_stats.columns[0:18]
away_cols = rolling_stats.columns[18:36]

In [22]:
# Create a dataframe where each column is equal to the home value minus the away value
##### This should be altered so that percentages are divided by each other instead of subtracted
for col in zip(stats_cols, home_cols, away_cols):
    rolling_stats_diff[col[0]] = rolling_stats[col[1]] - rolling_stats[col[2]]

In [23]:
# Rename the columns
rolling_stats_diff.columns = ["diff_" + col_name for col_name in rolling_stats_away.columns]

# putting it all together in 1 dataframe
rolling_stats = pd.merge(rolling_stats, rolling_stats_diff, on="game.id") # adding diff to the rolling stats
rolling_stats = pd.merge(labels, rolling_stats, on="game.id")             # adding labels

In [24]:
rolling_stats.head()

,game.date,game.season,winner,home_ast,home_blk,home_dreb,home_fg3_pct,home_fg3a,home_fg3m,home_fg_pct,home_fga,home_fgm,home_ft_pct,home_fta,home_ftm,home_oreb,home_pf,home_pts,home_reb,home_stl,home_turnover,away_ast,away_blk,away_dreb,away_fg3_pct,away_fg3a,away_fg3m,away_fg_pct,away_fga,away_fgm,away_ft_pct,away_fta,away_ftm,away_oreb,away_pf,away_pts,away_reb,away_stl,away_turnover,diff_away_ast,diff_away_blk,diff_away_dreb,diff_away_fg3_pct,diff_away_fg3a,diff_away_fg3m,diff_away_fg_pct,diff_away_fga,diff_away_fgm,diff_away_ft_pct,diff_away_fta,diff_away_ftm,diff_away_oreb,diff_away_pf,diff_away_pts,diff_away_reb,diff_away_stl,diff_away_turnover
game.id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2018-10-16,2018,1,22.15,3.50,30.55,0.186827,15.80,5.75,0.408467,83.50,37.25,0.488068,29.25,22.05,14.20,26.05,102.3,44.75,11.15,14.60,20.40,6.20,32.40,0.126757,11.50,3.95,0.454977,80.10,36.90,0.461474,27.80,20.35,12.55,20.85,98.1,44.95,8.65,14.95,1.75,-2.70,-1.85,0.060070,4.30,1.80,-0.046510,3.40,0.35,0.026595,1.45,1.70,1.65,5.20,4.2,-0.20,2.50,-0.35
2,2018-10-16,2018,1,24.65,4.35,28.95,0.128660,11.30,3.45,0.414999,85.70,37.70,0.468356,29.75,20.45,16.30,23.35,99.3,45.25,9.55,16.00,20.90,4.45,30.45,0.165451,14.50,5.55,0.423513,81.85,37.15,0.482440,25.05,17.65,13.20,20.30,97.5,43.65,7.55,15.40,3.75,-0.10,-1.50,-0.036791,-3.20,-2.10,-0.008514,3.85,0.55,-0.014083,4.70,2.80,3.10,3.05,1.8,1.60,2.00,0.60
3,2018-10-17,2018,0,25.70,6.40,31.20,0.105784,11.30,3.30,0.439629,75.15,34.85,0.511267,29.55,23.00,10.80,20.55,96.0,42.00,7.60,14.75,19.80,4.75,29.50,0.189570,17.35,7.10,0.415333,83.80,37.70,0.518994,25.60,20.80,12.70,27.85,103.3,42.20,7.80,13.30,5.90,1.65,1.70,-0.083786,-6.05,-3.80,0.024296,-8.65,-2.85,-0.007727,3.95,2.20,-1.90,-7.30,-7.3,-0.20,-0.20,1.45
4,2018-10-17,2018,1,19.15,5.55,31.05,0.133770,12.75,4.90,0.452937,77.10,35.15,0.412508,28.90,21.30,12.25,25.55,96.5,43.30,8.30,17.55,18.85,4.05,28.90,0.148624,11.80,3.80,0.382352,80.45,34.80,0.472927,25.45,19.20,10.25,23.10,92.6,39.15,9.25,13.70,0.30,1.50,2.15,-0.014854,0.95,1.10,0.070586,-3.35,0.35,-0.060419,3.45,2.10,2.00,2.45,3.9,4.15,-0.95,3.85
5,2018-10-17,2018,1,20.90,6.10,31.70,0.175094,15.35,5.60,0.393937,77.50,34.95,0.476045,26.80,21.10,10.70,23.25,96.6,42.40,7.50,13.05,21.95,3.55,28.85,0.127254,11.00,3.65,0.378380,81.10,34.20,0.408106,22.30,17.15,10.95,21.55,89.2,39.80,7.35,16.55,-1.05,2.55,2.85,0.047840,4.35,1.95,0.015556,-3.60,0.75,0.067940,4.50,3.95,-0.25,1.70,7.4,2.60,0.15,-3.50


In [25]:
rolling_stats.to_csv("/Users/tativalentine/Documents/GitHub/stats_feats.csv")